# <a name="how"></a> Как сделать пулл реквест?

0. Выбираем, где хотим провести изменения, в форке репозитория (более предпочтительно, но не принципиально) или в самом репозитории (в этом случае нужно запросить у меня доступ).
1. Нужно произвести все желаемые изменения в семинарском ноутбуке. И убедиться, что эти изменения сохранены (юпитер у меня иногда тупит, поэтому жму трижды `ctrl-s` с интервалом около секунды).
  <br> Постарайтесь ограничиться минимальными изменениями. Так же убедитесь, что у вас актуальная версия репозитория. (Я сам в недавние ноутбуки могу теоретически каждый день коммитить, а мои изменения затирать не надо :) ).
2. Далее в этом ноутбуке (он умеет правильно генерировать `.md` файлы):
  <br>A.  <a href="#what" style="color:#856024"> Здесь </a> выбираем семинар(ы), к которому сделали правку. `../tools` выбирать не надо.
  <br>B.  <a href="#github" style="color:#856024"> Здесь </a> можно написать свой commit message, если есть желание. Можно оставить как есть. В этом репозитории нет культуры хороших сообщений к коммитам :)
  <br>C.  Запускаем этот ноутбук, он сгенерит `.md`-шки и закоммитит изменения на гитхаб.
3. Если изменение было в форке, то делаем пулл реквест.

### <a name="what"></a> Выбираем что коммитить

In [12]:
import glob
import os
import subprocess

highlevel_dirs = sum([
    #["../tools"], 
    #sorted(glob.glob("../sem26*")),
    sorted(glob.glob("../sem17*")),
], [])

print("Highlevel dirs:", highlevel_dirs)

SyntaxError: invalid character in identifier (<ipython-input-12-f9dbabd7c183>, line 7)

In [5]:
tmp_dir = "./tmp_dir"
get_ipython().system('rm -r {tmp_dir} ; mkdir {tmp_dir} 2>&1 | grep -v "File exists"'.format(tmp_dir=tmp_dir))

### Генерируем все .md-шки стандартными средствами
\+ Делаем .md-шки очищенные для вывода. По этим .md-шкам можно будет смотреть реальную историю изменений. И дифф при пулреквестах.

In [6]:
from multiprocessing import Pool

tasks = []

def convert_tasks(n, d):
    no_output_file = d + "_no_output"
    src_copy = str(abs(hash(n))) + '_' + os.path.basename(n)
    path = os.path.dirname(n)
    return [
        "jupyter nbconvert {} --to markdown --output {}".format(n, d),
        " && ".join([
            "cp {src} {tmp_dir}/{src_copy}",
            "jupyter nbconvert {tmp_dir}/{src_copy} --ClearOutputPreprocessor.enabled=True --inplace",
            "jupyter nbconvert {tmp_dir}/{src_copy} --to markdown --output {src_copy}",
            "cp {tmp_dir}/{src_copy}.md {path}/{no_output_file}.md",
        ]).format(src=n, no_output_file=no_output_file, dst=d, tmp_dir=tmp_dir, src_copy=src_copy, path=path),
    ]
    
for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        tasks.extend(convert_tasks(notebooks[0], "README"))
    else:
        for n in notebooks:
            tasks.extend(convert_tasks(n, os.path.basename(n.replace(".ipynb", ""))))
        nmds = [os.path.basename(n).replace(".ipynb", ".md") for n in notebooks]
        with open(os.path.join(subdir, "README.md"), "w") as f:
            f.write('\n'.join(
                ['# Ноутбуки семинара'] + 
                ['* [{nmd}]({nmd})'.format(nmd=nmd) for nmd in nmds] + 
                ['']
            ))

print("\n".join(tasks))

def execute_all_in_parallel(tasks):
    ps = []
    for t in tasks:
        ps.append(subprocess.Popen(["bash", "-c", t], stdout=subprocess.PIPE, stderr=subprocess.PIPE))
    for p in ps:
        out, err = p.communicate()
        print(out.decode(), err.decode())

execute_all_in_parallel(tasks)

../sem26-fs-fuse ['../sem26-fs-fuse/fs_fuse.ipynb']
../sem27-python-bindings ['../sem27-python-bindings/bindings.ipynb']
jupyter nbconvert ../sem26-fs-fuse/fs_fuse.ipynb --to markdown --output README
cp ../sem26-fs-fuse/fs_fuse.ipynb ./tmp_dir/1677796532700934744_fs_fuse.ipynb && jupyter nbconvert ./tmp_dir/1677796532700934744_fs_fuse.ipynb --ClearOutputPreprocessor.enabled=True --inplace && jupyter nbconvert ./tmp_dir/1677796532700934744_fs_fuse.ipynb --to markdown --output 1677796532700934744_fs_fuse.ipynb && cp ./tmp_dir/1677796532700934744_fs_fuse.ipynb.md ../sem26-fs-fuse/README_no_output.md
jupyter nbconvert ../sem27-python-bindings/bindings.ipynb --to markdown --output README
cp ../sem27-python-bindings/bindings.ipynb ./tmp_dir/1633925075381849959_bindings.ipynb && jupyter nbconvert ./tmp_dir/1633925075381849959_bindings.ipynb --ClearOutputPreprocessor.enabled=True --inplace && jupyter nbconvert ./tmp_dir/1633925075381849959_bindings.ipynb --to markdown --output 1633925075381849

### Магические исправления

Стандартная конвертилка не учитывает некоторых особенностей маркдауна на гитхабе и некоторых особенностей моих ноутбуков. Поэтому результат надо допилить напильником.

In [7]:
import re


def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    return "dos2unix {}".format(fname)

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace("```python\n%%cmake", "```cmake\n%%cmake")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    #r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = re.sub(r'\<\!\-\-\ YANDEX_METRICA_BEGIN\ \-\-\>.*\<\!\-\-\ YANDEX_METRICA_END\ \-\-\>', '', r)
    
    r = r.replace("<IPython.core.display.Javascript object>", '')
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    template = "#""MAGICS_SETUP_END"
    bpos = r.rfind(template)
    if bpos != -1:
        r = r[bpos + len(template):]
        template = "```"
        bpos = r.find(template)
        assert bpos >= 0
        r = r[bpos + len(template):]
    
    
    template = "<""!-- MAGICS_SETUP_PRINTING_END -->"
    bpos = r.rfind(template)
    if bpos != -1:
        r = r[bpos + len(template):]
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if fname.find("__FILE__") == -1:
            with open(fname, "r") as f:
                return "\n```\n" + f.read() + "\n```\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    if fname.endswith(".md"):
        improve_md(fname)


In [8]:
tasks = []
shell_tasks = []


for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            shell_tasks.append(basic_improve(fname))
            tasks.append(lambda fname=fname: improve_file(fname))
            
execute_all_in_parallel(shell_tasks)
for t in tasks:
    t()

 dos2unix: converting file ./../sem26-fs-fuse/fs_fuse.ipynb to Unix format...

 dos2unix: converting file ./../sem27-python-bindings/bindings.ipynb to Unix format...

 dos2unix: converting file ./../sem26-fs-fuse/README.md to Unix format...

 dos2unix: converting file ./../sem26-fs-fuse/README_no_output.md to Unix format...

 dos2unix: converting file ./../sem27-python-bindings/README.md to Unix format...

 dos2unix: converting file ./../sem27-python-bindings/README_no_output.md to Unix format...



### Смотрим изменения

In [9]:
for subdir in highlevel_dirs:
    get_ipython().system("git diff {}/*_no_output*".format(subdir))

diff --git a/sem26-fs-fuse/README_no_output.md b/sem26-fs-fuse/README_no_output.md
index eb07774..f7d11e0 100644
--- a/sem26-fs-fuse/README_no_output.md
+++ b/sem26-fs-fuse/README_no_output.md
@@ -192,10 +192,13 @@ int main(int argc, char** argv) {
 
 ```
 
+# FUSE
 
-```python
+Важные опции
+* `-f` - запуск в синхронном режиме (без этой опции будет создан демон, а сама программа почти сразу завершится)
+* `-s` - запуск в однопоточном режиме.
 
-```
+В этом месте что-нибудь про демонизацию стоит расскзать, наверное.
 
 ## <a name="fusepy"></a> Python + fusepy
 
diff --git a/sem27-python-bindings/README_no_output.md b/sem27-python-bindings/README_no_output.md
index 7cd90ce..34ef1b8 100644
--- a/sem27-python-bindings/README_no_output.md
+++ b/sem27-python-bindings/README_no_output.md
@@ -207,7 +207,7 @@ static PyObject* print_dict(PyObject* self, PyObject* args, PyObject* kwargs) {
     printf("\n");
     fflush(stdout);
 
-    Py_RETURN_NONE;
+    Py_RETURN_NONE; // Инкрементит счетчик 

### <a name="github"></a> Коммитим на github

In [11]:
cmds = []
add_cmd = "git add --ignore-errors "
add_cmd_f = "git add --ignore-errors -f "
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        cmds.append(add_cmd + " {}/*{}".format(subdir, sfx))
    cmds.append(add_cmd_f + " -f {}/bash_popen_tmp/*.html".format(subdir))
    cmds.append(add_cmd_f + " -f {}/interactive_launcher_tmp/*.log".format(subdir))
    cmds.append("git add -u {}".format(subdir))
    
def execute_cmd(cmd):
    print(">", cmd)
    get_ipython().system(cmd)
    
for cmd in cmds:
    execute_cmd(cmd)
# execute_cmd("git add -u")
execute_cmd("git commit -m 'yet another update'")
execute_cmd("git push origin master")

> git add --ignore-errors  ../sem26-fs-fuse/*.ipynb
> git add --ignore-errors  ../sem26-fs-fuse/*.md
> git add --ignore-errors  ../sem26-fs-fuse/*.c
> git add --ignore-errors  ../sem26-fs-fuse/*.cpp
> git add --ignore-errors -f  -f ../sem26-fs-fuse/bash_popen_tmp/*.html
fatal: pathspec '../sem26-fs-fuse/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem26-fs-fuse/interactive_launcher_tmp/*.log
> git add -u ../sem26-fs-fuse
> git add --ignore-errors  ../sem27-python-bindings/*.ipynb
> git add --ignore-errors  ../sem27-python-bindings/*.md
> git add --ignore-errors  ../sem27-python-bindings/*.c
> git add --ignore-errors  ../sem27-python-bindings/*.cpp
> git add --ignore-errors -f  -f ../sem27-python-bindings/bash_popen_tmp/*.html
fatal: pathspec '../sem27-python-bindings/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem27-python-bindings/interactive_launcher_tmp/*.log
fatal: pathspec '../sem27-python-bindings/intera